In [1]:
# 절대 임포트 설정
from __future__ import division, print_function, absolute_import

# 필요한 라이브러리들을 임포트
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.contrib import rnn
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# 파라미터 설정
learning_rate_RMSProp = 0.01
learning_rate_Gradient_Descent = 0.5
training_epochs = 50     # epoch 횟수 (iteration)
softmax_classifier_iterations = 1000 # Softmax Classifier iteration 횟수 
batch_size = 256          
display_step = 1        # 몇 Step마다 log를 출력할지 결정한다.
examples_to_show = 10   # reconstruct된 이미지 중 몇개를 보여줄지를 결정한다. 
       
param_sparsity = 0.01
param_weight_decay = 0.0001
param_sparse_panelty = 3

In [4]:
def regularization(weights):
    return tf.nn.l2_loss(weights)

def Single_Layer_Autoencoder(Input_Layer, n_input, n_hidden):
    with tf.name_scope("Hidden_Layer") as scope:
        # 히든 레이어 1을 위한 Weights와 Biases
        W_ih = tf.Variable(tf.random_normal([n_input, n_hidden]), name = "Weight_IH")  # IH = Input_Hidden
        b_ih = tf.Variable(tf.random_normal([n_hidden]), name = "Bias_IH")
        H_Layer = tf.nn.sigmoid(tf.matmul(Input_Layer, W_ih) + b_ih, name = "Hidden")     # 히든레이어 1의 activation (sigmoid 함수를 사용)
               
    with tf.name_scope("Reconstructed_Layer") as scope:
        # Output 레이어를 위한 Weights와 Biases
        W_hr = tf.Variable(tf.random_normal([n_hidden, n_input]), name = "Weight_HR")  # HR = Hidden_Reconstructed
        b_hr = tf.Variable(tf.random_normal([n_input]), name = "Bias_HR")
        X_reconstructed = tf.nn.sigmoid(tf.matmul(H_Layer,W_hr) + b_hr, name = "Hidden_HR")   # Output 레이어의 activation (sigmoid 함수를 사용)
    
    with tf.name_scope("cost") as scope:        
    # Autoencoder Optimization을 위한 파라미터들
        average_act_hidden = tf.reduce_mean(H_Layer,axis=0)   #Average hidden layer over all data points in X, Page 14 in https://web.stanford.edu/class/cs294a/sparseAutoencoder_2011new.pdf
        diff = Input_Layer - X_reconstructed
        KL = Kullback_Leibler_divergence(rho, average_act_hidden)
        cost= 0.5*tf.reduce_mean(tf.reduce_sum(tf.pow(diff,2),axis=1)) + 0.5*Lambda*(tf.nn.l2_loss(W_ih) + tf.nn.l2_loss(W_hr)) + Beta*tf.reduce_sum(KL)
        cost_summary = tf.summary.scalar("cost", cost)
        

    return H_Layer, X_reconstructed, cost

def Kullback_Leibler_divergence(rho, rho_hat):
    return rho * tf.log(rho) - rho * tf.log(rho_hat) + (1 - rho) * tf.log(1 - rho) - (1 - rho) * tf.log(1 - rho_hat)

def LSTM(x_input, stateSize, n_classes, y):
    with tf.name_scope("LSTM_graph") as scope:
        
        cellState = tf.placeholder(tf.float32, [batch_size, stateSize])
        hiddenState = tf.placeholder(tf.float32, [batch_size, stateSize])
        initState = rnn.LSTMStateTuple(cellState, hiddenState)
        
        W = tf.Variable(np.random.rand(stateSize, n_classes), dtype=tf.float32, name="weight1")
        bias1 = tf.Variable(np.zeros((1, n_classes)), dtype=tf.float32)
        W2 = tf.Variable(np.random.rand(stateSize, n_classes), dtype=tf.float32, name="weight2")
        bias2 = tf.Variable(np.zeros((1, n_classes)), dtype=tf.float32)
        tf.summary.histogram(name="weights", values=W)
        #weights = tf.Variable(tf.random_normal([cell_size, n_classes]))
        #biases = tf.Variable(tf.random_normal([n_classes]))
        lstm_cell = rnn.BasicLSTMCell(stateSize, forget_bias=1.0, state_is_tuple=True)
        #outputs, states = tf.nn.static_rnn(lstm_cell, x_input, dtype=tf.float32)
        stateSeries, currentState = rnn.static_rnn(lstm_cell, x_input, initState)
        
        # calculate loss
        logits_series = [tf.matmul(state, W2) + bias2 for state in stateSeries]
        prediction_series = [tf.nn.softmax(logits) for logits in logits_series]
        
        #losses = [tf.nn.sparse_softmax_cross_entropy_with_logits(labels = labels, logits=logits) for logits, labels in zip(logits_series, y)]
        series_stack = tf.stack([tf.transpose(tf.reshape(logits_series, [batch_size, n_classes])), tf.transpose(y, name='Transpose_y')], axis=2, name="logit_label_stacks")
        losses = tf.map_fn(lambda logits, labels: tf.nn.sparse_softmax_cross_entropy_with_logits(labels = labels, logits=logits))
        total_loss = tf.reduce_mean(losses, name="total_loss")
        
        #pred = tf.matmul(outputs[-1], weights) + biases
        #cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(logits = pred, labels = y))
            
    return total_loss, prediction_series, currentState

In [ ]:
rho = param_sparsity
Lambda = param_weight_decay
Beta = param_sparse_panelty

# 학습에 필요한 변수들 설정
with tf.name_scope("Tensor_initialize"):
    X = tf.placeholder("float", [None, 784], name = "Input")    # Input 데이터 설정
    y_ = tf.placeholder(tf.float32, [None, 10], name = "True_Output")   # True Output
    
    batch_X = tf.placeholder("float", [None, 784], name = "Input")    # Input 데이터 설정
    batch_y_ = tf.placeholder(tf.float32, [None, 10], name = "True_Output")   # True Output

with tf.name_scope("Auto_Encoder_01"):
    H1, O1, cost1 = Single_Layer_Autoencoder(X, 784, 400)
    optmzr1 = tf.train.GradientDescentOptimizer(learning_rate_Gradient_Descent).minimize(cost1)

with tf.name_scope("Auto_Encoder_02"):
    H2, O2, cost2 = Single_Layer_Autoencoder(H1, 400, 400)
    optmzr2 = tf.train.GradientDescentOptimizer(learning_rate_Gradient_Descent).minimize(cost2)

with tf.name_scope("Auto_Encoder_03"):
    H3, O3, cost3 = Single_Layer_Autoencoder(H2, 400, 400)
    optmzr3 = tf.train.GradientDescentOptimizer(learning_rate_Gradient_Descent).minimize(cost3)

with tf.name_scope("Auto_Encoder_04"):
    H4, O4, cost4 = Single_Layer_Autoencoder(H3, 400, 400)
    optmzr4 = tf.train.GradientDescentOptimizer(learning_rate_Gradient_Descent).minimize(cost4)

with tf.name_scope("LSTM"):
    lstm_X = tf.split(H4, 1, 0)
    lstm_cost, lstm_pred, lstm_current = LSTM(lstm_X, 5, 10, y_)
    lstm_optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(lstm_cost)
    correct_pred = tf.equal(tf.argmax(lstm_pred, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [5]:

    
from tqdm import tqdm_notebook
from pywt import wavedec, waverec

with tf.Session() as sess:
    
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./logs/SAE_test7")
    writer.add_graph(sess.graph)

    # 변수들을 초기화한다.
    sess.run(tf.global_variables_initializer())
    
    # Step 1: Stacked Autoencoder pre-training 
    total_batch = int(mnist.train.num_examples/batch_size)
    print("total_batch ", total_batch)
    # Training을 시작한다.
    for epoch in tqdm_notebook(range(training_epochs)):
        # 모든 배치들을 돌아가면서(Loop) 학습한다.
        _current_cell_state = np.zeros((batch_size, stateSize))
        _current_hidden_state = np.zeros((batch_size, stateSize))
        print("New data, epoch", epoch)
        
        for batch_idx in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            
            coeffs = wavedec(batch_xs, 'haar', level=2)
            reconstruct_xs = waverec(coeffs, 'haar')
            
            # batch 데이터를 이용해서 트레이닝을 진행한다.
            
            # Stacked Auto-encoder
            _, SAE_cost1 = sess.run([optmzr1, cost1], feed_dict={X: reconstruct_xs, y_: batch_ys})
            _, SAE_cost2 = sess.run([optmzr2, cost2], feed_dict={X: reconstruct_xs, y_: batch_ys})
            _, SAE_cost3 = sess.run([optmzr3, cost3], feed_dict={X: reconstruct_xs, y_: batch_ys})
            _, SAE_cost4 = sess.run([optmzr4, cost4], feed_dict={X: reconstruct_xs, y_: batch_ys})                       
            
            _total_loss, _train_step, _current_state, _predictions_series = sess.run([lstm_cost, lstm_optimizer, lstm_current, lstm_pred], feed_dict={X:reconstruct_xs})
                        
            summary = sess.run(merged_summary, feed_dict={X: reconstruct_xs, y_: batch_ys})
            writer.add_summary(summary, batch_idx)
            
        # 일정 epoch step마다 로그를 출력한다.
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "SAE cost1=", "{:.9f}".format(SAE_cost1), "SAE cost2=", "{:.9f}".format(SAE_cost2), "SAE cost3=", "{:.9f}".format(SAE_cost3), "SAE cost4=", "{:.9f}".format(SAE_cost4), "LSTM loss=", "{:.9f}".format(_total_loss))
    print("Stacked Autoencoder + LSTM training Optimization Finished!")

    # Step 2: test 데이터셋을 autoencoder로 reconstruction 해본다.
    reconstructed_image1 = sess.run(O1, feed_dict={X: mnist.test.images[:examples_to_show]})
    reconstructed_image2 = sess.run(O2, feed_dict={X: mnist.test.images[:examples_to_show]})
    reconstructed_image3 = sess.run(O3, feed_dict={X: mnist.test.images[:examples_to_show]})
    hidden_image1 = sess.run(H1, feed_dict={X: mnist.test.images[:examples_to_show]})
    hidden_image2 = sess.run(H2, feed_dict={X: mnist.test.images[:examples_to_show]})
    hidden_image3 = sess.run(H3, feed_dict={X: mnist.test.images[:examples_to_show]})
    

    # 원본 이미지와 재구축(reconstructed)된 이미지를 비교한다.
    f, a = plt.subplots(7, 10, figsize=(10, 4))
    for i in range(examples_to_show):
        a[0][i].imshow(np.reshape(mnist.test.images[i], (28, 28)))
        a[1][i].imshow(np.reshape(hidden_image1[i], (20, 20)))
        a[2][i].imshow(np.reshape(reconstructed_image1[i], (28, 28)))        
        a[3][i].imshow(np.reshape(hidden_image2[i], (20, 20)))
        a[4][i].imshow(np.reshape(reconstructed_image2[i], (20, 20)))
        a[5][i].imshow(np.reshape(hidden_image3[i], (20, 20)))
        a[6][i].imshow(np.reshape(reconstructed_image3[i], (20, 20)))
    
    plt.draw()

    #plt.waitforbuttonpress()     # 버튼을 누를때까지 작업 정지 
    f.savefig('reconstructed_mnist_image.png')  # reconstruction 결과를 png로 저장한다.    
 

ValueError: Dimension 1 in both shapes must be equal, but are 256 and 1. Shapes are [10,256] and [10,1].
	From merging shape 0 with other shapes. for 'LSTM/LSTM_graph/logit_label_stacks' (op: 'Pack') with input shapes: [10,256], [10,1].